In [1]:
// libTorch 1.5
// Load all LibTorch dependencies for cling
#pragma cling add_library_path("/Users/zqfang/Github/XCpp/libtorch/lib")
#pragma cling add_include_path("/Users/zqfang/Github/XCpp/libtorch/include/")
#pragma cling add_include_path("/Users/zqfang/Github/XCpp/libtorch/include/torch/csrc/api/include/")

// Note: This is loaded for macOS environment, for Linux try loading *.so files
#pragma cling load("/Users/zqfang/Github/XCpp/libtorch/lib/libc10.dylib")
#pragma cling load("/Users/zqfang/Github/XCpp/libtorch/lib/libiomp5.dylib")
#pragma cling load("/Users/zqfang/Github/XCpp/libtorch/lib/libtorch_cpu.dylib")
#pragma cling load("/Users/zqfang/Github/XCpp/libtorch/lib/libtorch.dylib")
#pragma cling load("/Users/zqfang/Github/XCpp/libtorch/lib/libtorch_global_deps.dylib")

#pragma cling load("/Users/zqfang/Github/XCpp/libtorch/lib/libfbjni.dylib")
#pragma cling load("/Users/zqfang/Github/XCpp/libtorch/lib/libpytorch_jni.dylib")
#pragma cling load("/Users/zqfang/Github/XCpp/libtorch/lib/libtorch_python.dylib")
#pragma cling load("/Users/zqfang/Github/XCpp/libtorch/lib/libshm.dylib")

#pragma cling load("/Users/zqfang/Github/XCpp/libtorch/lib/libcaffe2_detectron_ops.dylib")
#pragma cling load("/Users/zqfang/Github/XCpp/libtorch/lib/libcaffe2_module_test_dynamic.dylib")
#pragma cling load("/Users/zqfang/Github/XCpp/libtorch/lib/libcaffe2_observers.dylib")

cling::DynamicLibraryManager::loadLibrary(): dlopen(/Users/zqfang/Github/XCpp/libtorch/lib/libtorch_cpu.dylib, 9): Library not loaded: @rpath/libiomp5.dylib
  Referenced from: /Users/zqfang/Github/XCpp/libtorch/lib/libtorch_cpu.dylib
  Reason: image not found
cling::DynamicLibraryManager::loadLibrary(): dlopen(/Users/zqfang/Github/XCpp/libtorch/lib/libtorch.dylib, 9): Library not loaded: @rpath/libtorch_cpu.dylib
  Referenced from: /Users/zqfang/Github/XCpp/libtorch/lib/libtorch.dylib
  Reason: image not found
cling::DynamicLibraryManager::loadLibrary(): dlopen(/Users/zqfang/Github/XCpp/libtorch/lib/libtorch_global_deps.dylib, 9): Library not loaded: @rpath/libiomp5.dylib
  Referenced from: /Users/zqfang/Github/XCpp/libtorch/lib/libtorch_global_deps.dylib
  Reason: image not found
cling::DynamicLibraryManager::loadLibrary(): dlopen(/Users/zqfang/Github/XCpp/libtorch/lib/libtorch_python.dylib, 9): Library not loaded: @rpath/libshm.dylib
  Referenced from: /Users/zqfang/Github/XCpp/libto

In [2]:
#include <torch/torch.h>
#include <torch/script.h>
#include <iostream>
#include <iomanip>

void print_tensor_size(const torch::Tensor&);
void print_script_module(const torch::jit::script::Module& module, size_t spaces = 0);

In [3]:
std::cout << "PyTorch Basics\n\n";

// Set floating point output precision
std::cout << std::fixed << std::setprecision(4);

PyTorch Basics



In [4]:
std::cout << "---- BASIC AUTOGRAD EXAMPLE 1 ----\n";

// Create Tensors
torch::Tensor x = torch::tensor(1.0, torch::requires_grad());
torch::Tensor w = torch::tensor(2.0, torch::requires_grad());
torch::Tensor b = torch::tensor(3.0, torch::requires_grad());

// Build a computational graph
auto y = w * x + b;  // y = 2 * x + 3

// Compute the gradients
y.backward();

// Print out the gradients
std::cout << x.grad() << '\n';  // x.grad() = 2
std::cout << w.grad() << '\n';  // w.grad() = 1
std::cout << b.grad() << "\n\n";  // b.grad() = 1


---- BASIC AUTOGRAD EXAMPLE 1 ----
2
[ CPUFloatType{} ]
1
[ CPUFloatType{} ]
1
[ CPUFloatType{} ]



@0x107267ed0

In [5]:
std::cout << "---- BASIC AUTOGRAD EXAMPLE 2 ----\n";

// Create Tensors of shapes
x = torch::randn({10, 3});
y = torch::randn({10, 2});

// Build a fully connected layer
torch::nn::Linear linear(3, 2);
std::cout << "w:\n" << linear->weight << '\n';
std::cout << "b:\n" << linear->bias << '\n';

// Create loss function and optimizer
torch::nn::MSELoss criterion;
torch::optim::SGD optimizer(linear->parameters(), torch::optim::SGDOptions(0.01));

// Forward pass
auto pred = linear->forward(x);

// Compute loss
auto loss = criterion(pred, y);
std::cout << "Loss: " << loss.item<double>() << '\n';

// Backward pass
loss.backward();

// Print out the gradients
std::cout << "dL/dw:\n" << linear->weight.grad() << '\n';
std::cout << "dL/db:\n" << linear->bias.grad() << '\n';

// 1 step gradient descent
optimizer.step();

// Print out the loss after 1-step gradient descent
pred = linear->forward(x);
loss = criterion(pred, y);
std::cout << "Loss after 1 optimization step: " << loss.item<double>() << "\n\n";


---- BASIC AUTOGRAD EXAMPLE 2 ----
w:
-0.2216 -0.0401 -0.0063
 0.5274 -0.2586  0.5540
[ CPUFloatType{2,3} ]
b:
-0.2209
 0.1046
[ CPUFloatType{2} ]
Loss: 2.0071
dL/dw:
-0.3324  0.4685  0.7034
 0.7355 -0.7133  0.9054
[ CPUFloatType{2,3} ]
dL/db:
-0.4277
-0.3347
[ CPUFloatType{2} ]
Loss after 1 optimization step: 1.9774



In [6]:
std::cout << "---- CREATING TENSORS FROM EXISTING DATA ----\n";

// WARNING: Tensors created with torch::from_blob(ptr_to_data, ...) do not own
// the memory pointed to by ptr_to_data!
// (see https://pytorch.org/cppdocs/notes/tensor_basics.html#using-externally-created-data)
//
// If you want a tensor that has its own copy of the data you can call clone() on the
// tensor returned from torch::from_blob(), e.g.:
// torch::Tensor t = torch::from_blob(ptr_to_data, ...).clone();
// (see https://github.com/pytorch/pytorch/issues/12506#issuecomment-429573396)

// Tensor From C-style array
float data_array[] = {1, 2, 3, 4};
torch::Tensor t1 = torch::from_blob(data_array, {2, 2});
std::cout << "Tensor from array:\n" << t1 << '\n';

TORCH_CHECK(data_array == t1.data_ptr<float>());

// Tensor from vector:
std::vector<float> data_vector = {1, 2, 3, 4};
torch::Tensor t2 = torch::from_blob(data_vector.data(), {2, 2});
std::cout << "Tensor from vector:\n" << t2 << "\n\n";

TORCH_CHECK(data_vector.data() == t2.data_ptr<float>());

---- CREATING TENSORS FROM EXISTING DATA ----
Tensor from array:
 1  2
 3  4
[ CPUFloatType{2,2} ]
Tensor from vector:
 1  2
 3  4
[ CPUFloatType{2,2} ]



# Indexing

In [7]:
std::cout << "---- SLICING AND EXTRACTING PARTS FROM TENSORS ----\n";

std::vector<int64_t> test_data = {1, 2, 3, 4, 5, 6, 7, 8, 9};
torch::Tensor s = torch::from_blob(test_data.data(), {3, 3}, torch::kInt64);
std::cout << "s:\n" << s << '\n';
// Output:
// 1 2 3
// 4 5 6
// 7 8 9

---- SLICING AND EXTRACTING PARTS FROM TENSORS ----
s:
 1  2  3
 4  5  6
 7  8  9
[ CPULongType{3,3} ]


@0x107267ed0

In [8]:
// using torch::indexing::Slice;
// using torch::indexing::None;
// using torch::indexing::Ellipsis;
using namespace torch::indexing;

In [9]:
// Indexing and slicing tensors is done very similarly to how
// it is done in Python.
//
// For a complete side-by-side translation of all indexing types, see:
// https://pytorch.org/cppdocs/notes/tensor_indexing.html

// Extract a single element tensor:
std::cout << "\"s[0,2]\" as tensor:\n" << s.index({0, 2}) << '\n';
std::cout << "\"s[0,2]\" as value:\n" << s.index({0, 2}).item<int64_t>() << '\n';
// Output:
// 3

// Slice a tensor along a dimension at a given index.
std::cout << "\"s[:,2]\":\n" << s.index({Slice(), 2}) << '\n';
// Output:
// 3
// 6
// 9

// Slice a tensor along a dimension at given indices from
// a start-index up to - but not including - an end-index using a given step size.
std::cout << "\"s[:2,:]\":\n" << s.index({Slice(None, 2), Slice()}) << '\n';
// Output:
// 1 2 3
// 4 5 6
std::cout << "\"s[:,1:]\":\n" << s.index({Slice(), Slice(1, None)}) << '\n';
// Output:
// 2 3
// 5 6
// 8 9
std::cout << "\"s[:,::2]\":\n" << s.index({Slice(), Slice(None, None, 2)}) << '\n';
// Output:
// 1 3
// 4 6
// 7 9

// Combination.
std::cout << "\"s[:2,1]\":\n" << s.index({Slice(None, 2), 1}) << '\n';
// Output:
// 2
// 5

// Ellipsis (...).
std::cout << "\"s[..., :2]\":\n" << s.index({Ellipsis, Slice(None, 2)}) << "\n\n";
// Output:
// 1 2
// 4 5
// 7 8


"s[0,2]" as tensor:
3
[ CPULongType{} ]
"s[0,2]" as value:
3
"s[:,2]":
 3
 6
 9
[ CPULongType{3} ]
"s[:2,:]":
 1  2  3
 4  5  6
[ CPULongType{2,3} ]
"s[:,1:]":
 2  3
 5  6
 8  9
[ CPULongType{3,2} ]
"s[:,::2]":
 1  3
 4  6
 7  9
[ CPULongType{3,2} ]
"s[:2,1]":
 2
 5
[ CPULongType{2} ]
"s[..., :2]":
 1  2
 4  5
 7  8
[ CPULongType{3,2} ]



@0x107267ed0

In [ ]:
// =============================================================== //
//                         INPUT PIPELINE                          //
// =============================================================== //

std::cout << "---- INPUT PIPELINE ----\n";

// Construct MNIST dataset
const std::string MNIST_data_path = "data/mnist/";

auto dataset = torch::data::datasets::MNIST(MNIST_data_path)
    .map(torch::data::transforms::Normalize<>(0.1307, 0.3081))
    .map(torch::data::transforms::Stack<>());

// Fetch one data pair
auto example = dataset.get_batch(0);
std::cout << "Sample data size: ";
std::cout << example.data.sizes() << "\n";
std::cout << "Sample target: " << example.target.item<int>() << "\n";

// Construct data loader
auto dataloader = torch::data::make_data_loader<torch::data::samplers::RandomSampler>(
    dataset, 64);

// Fetch a mini-batch
auto example_batch = *dataloader->begin();
std::cout << "Sample batch - data size: ";
std::cout << example_batch.data.sizes() << "\n";
std::cout << "Sample batch - target size: ";
std::cout << example_batch.target.sizes() << "\n\n";
// Actual usage of the dataloader:
// for (auto& batch : *dataloader) {
    // Training code here
// }